In [25]:
import pandas as pd 
import numpy as np
import requests
import re
import os
import time
import json

archivo = open("C:/Users/Nur/Documents/tokens/tmdb_token.txt", "r")
contenido = archivo.read()
archivo.close()
token = contenido

In [26]:
df_best_picture = pd.read_csv("./data/best_picture.csv")
df_best_picture

year_film  year_ceremony  ceremony  \
0         1927           1928         1   
1         1927           1928         1   
2         1927           1928         1   
3         1928           1929         2   
4         1928           1929         2   
..         ...            ...       ...   
586       2022           2023        95   
587       2022           2023        95   
588       2022           2023        95   
589       2022           2023        95   
590       2022           2023        95   

                                                  name                 film  \
0                                    The Caddo Company           The Racket   
1                                                  Fox           7th Heaven   
2                               Paramount Famous Lasky                Wings   
3                                  Feature Productions                Alibi   
4                                                  Fox       In Old Arizona   
..                                                 ...                  ...   
586  Kristie Macosko Krieger, Steven Spielberg and ...        The Fabelmans   
587  Todd Field, Alexandra Milchan and Scott Lamber...                  Tár   
588  Tom Cruise, Christopher McQuarrie, David Ellis...    Top Gun: Maverick   
589     Erik Hemmendorff and Philippe Bober, Producers  Triangle of Sadness   
590  Dede Gardner, Jeremy Kleiner and Frances McDor...        Women Talking   

     winner  
0     False  
1     False  
2      True  
3     False  
4     False  
..      ...  
586   False  
587   False  
588   False  
589   False  
590   False  

[591 rows x 6 columns]

# INFO

- [ ] cruzar con tmdb, rotten tomatoes, etc
- [ ] oscar bait
- [ ] [IMDB academy awards](https://www.imdb.com/event/ev0000003/2023/1/?ref_=ev_eh)
- [ ] [Cinema timeline](https://www.timetoast.com/timelines/evolution-of-movies-75835972-1771-4afa-8a8c-6b3356decbf6)
- [ ] [AI Movie Companion for Ratings](https://routinehub.co/shortcut/15932/)

# PRUEBAS

[TMDB API](https://developer.themoviedb.org/reference/search-movie)

In [27]:
prueba = pd.DataFrame(df_best_picture.loc[587]).T
prueba

year_film year_ceremony ceremony  \
587      2022          2023       95   

                                                  name film winner  
587  Todd Field, Alexandra Milchan and Scott Lamber...  Tár  False

In [28]:
movie_data = []
base_url = "https://api.themoviedb.org/3/search/movie?"

batches = np.array_split(df_best_picture, 11)  # Dividir en 10 lotes (ajusta según sea necesario)
for batch in batches:
    for index, row in batch.iterrows():
        # Obtener el nombre de la película y el año de la fila actual
        name = row['film'].replace(" ", "%20")
        year = row['year_film']

        # Construir la URL con los parámetros de la búsqueda
        url = f"{base_url}query={name}&include_adult=false&language=en-US&page=1&year={year}"

        # Configurar los encabezados con el token de autenticación
        headers = {
            "accept": "application/json",
            "Authorization": f"Bearer {token}"
        }

        # Realizar la solicitud a la API
        response = requests.get(url, headers=headers)

        # Buscar la primera coincidencia exacta en los resultados
        found = False
        for result in response.json()['results']:
            if result['title'].lower() == row['film'].lower():
                movie_data.append(result)
                found = True
                break  # Salir del bucle después de encontrar la primera coincidencia

        # Manejar el caso en el que no se encuentre ninguna coincidencia
        if not found:
            year = row['year_film']+1

            url = f"{base_url}query={name}&include_adult=false&language=en-US&page=1&year={year}"

            headers = {
                "accept": "application/json",
                "Authorization": f"Bearer {token}"
            }
            response = requests.get(url, headers=headers)

            found = False
            for result in response.json()['results']:
                if result['title'].lower() == row['film'].lower():
                    movie_data.append(result)
                    found = True
                    break
            
            if not found:
                year = row['year_film']-1

                url = f"{base_url}query={name}&include_adult=false&language=en-US&page=1&year={year}"

                headers = {
                    "accept": "application/json",
                    "Authorization": f"Bearer {token}"
                }
                response = requests.get(url, headers=headers)

                found = False
                for result in response.json()['results']:
                    if result['title'].lower() == row['film'].lower():
                        movie_data.append(result)
                        found = True
                        break
                if not found:
                    movie_data.append({"title": f"{row['film']}",
                                    "release_date": f"{row['year_film']}"})

    # Ahora movie_data contiene la información de todas las películas en df_best_picture


c:\Users\Nur\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\core\fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


In [29]:
movie_data

[{'adult': False,
  'backdrop_path': '/yCpYVgJH7d49vDc44SKrHLEVhcq.jpg',
  'genre_ids': [80, 18],
  'id': 27777,
  'original_language': 'en',
  'original_title': 'The Racket',
  'overview': 'A renegade police captain sets out to catch a sadistic mob boss.',
  'popularity': 1.882,
  'poster_path': '/wHAJfHslSb6OpNQmj1hu9aE9IQR.jpg',
  'release_date': '1928-11-01',
  'title': 'The Racket',
  'video': False,
  'vote_average': 6.4,
  'vote_count': 29},
 {'adult': False,
  'backdrop_path': '/ebgjvr2dwubTYzOfa419bAzumQ0.jpg',
  'genre_ids': [18, 10749],
  'id': 82474,
  'original_language': 'en',
  'original_title': '7th Heaven',
  'overview': 'A dejected Parisian sewer worker feels his prayers have been answered when he falls in love with a street waif.',
  'popularity': 6.397,
  'poster_path': '/3ETXMu5PrcVMRJpfN2Z3MwcfXzy.jpg',
  'release_date': '1927-09-10',
  'title': '7th Heaven',
  'video': False,
  'vote_average': 7.317,
  'vote_count': 90},
 {'adult': False,
  'backdrop_path': '/soJ

In [30]:
df_movie_data = pd.DataFrame(movie_data)
df_movie_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 591 entries, 0 to 590
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   adult              591 non-null    bool   
 1   backdrop_path      591 non-null    object 
 2   genre_ids          591 non-null    object 
 3   id                 591 non-null    int64  
 4   original_language  591 non-null    object 
 5   original_title     591 non-null    object 
 6   overview           591 non-null    object 
 7   popularity         591 non-null    float64
 8   poster_path        591 non-null    object 
 9   release_date       591 non-null    object 
 10  title              591 non-null    object 
 11  video              591 non-null    bool   
 12  vote_average       591 non-null    float64
 13  vote_count         591 non-null    int64  
dtypes: bool(2), float64(2), int64(2), object(8)
memory usage: 56.7+ KB


In [31]:
df_movie_data

adult                     backdrop_path               genre_ids      id  \
0    False  /yCpYVgJH7d49vDc44SKrHLEVhcq.jpg                [80, 18]   27777   
1    False  /ebgjvr2dwubTYzOfa419bAzumQ0.jpg             [18, 10749]   82474   
2    False  /soJGAjMOnvHaWZvAZm7r5OMTpM4.jpg  [18, 28, 10752, 10749]   28966   
3    False  /oWD7JsI3mdJ8A4fZTs5GymqoeOr.jpg            [28, 80, 53]   13847   
4    False   /VRjzMGnGuAXycd1alavkwgAqQJ.jpg                [28, 37]   83989   
..     ...                               ...                     ...     ...   
586  False  /2OnmtVOEpCwt0sYtss5kNxyYM2W.jpg                [18, 35]  804095   
587  False  /84XcRwKHAw4VXdKOYTSW5ARxFEt.jpg             [18, 10402]  817758   
588  False  /AaV1YIdWKnjAIAOe8UUKBFm327v.jpg                [28, 18]  361743   
589  False  /jENycYiYGr5NvMs789zj06JkW5l.jpg                [35, 18]  497828   
590  False  /hUtrCoGrlo3o8juAI1ZzA7F1LX9.jpg                [18, 80]  777245   

    original_language       original_title  \
0                  en           The Racket   
1                  en           7th Heaven   
2                  en                Wings   
3                  en                Alibi   
4                  en       In Old Arizona   
..                ...                  ...   
586                en        The Fabelmans   
587                en                  TÁR   
588                en    Top Gun: Maverick   
589                en  Triangle of Sadness   
590                en        Women Talking   

                                              overview  popularity  \
0    A renegade police captain sets out to catch a ...       1.882   
1    A dejected Parisian sewer worker feels his pra...       6.397   
2    Two young men, one rich, one middle class, bot...      10.823   
3    Chick Williams, a prohibition gangster, rejoin...       1.400   
4    Army Sergeant Mickey Dunn sets out in pursuit ...       3.483   
..                                                 ...         ...   
586  Growing up in post-World War II era Arizona, y...      53.009   
587  Renowned musician Lydia Tár is days away from ...      38.118   
588  After more than thirty years of service as one...     204.670   
589  A celebrity model couple are invited on a luxu...      50.330   
590  A group of women in an isolated religious colo...      19.525   

                          poster_path release_date                title  \
0    /wHAJfHslSb6OpNQmj1hu9aE9IQR.jpg   1928-11-01           The Racket   
1    /3ETXMu5PrcVMRJpfN2Z3MwcfXzy.jpg   1927-09-10           7th Heaven   
2    /kEl6KCBgdmT1Nex3ka0EIWAOmtm.jpg   1927-08-12                Wings   
3    /2C3ZDxVrOVafzNoOVrxsNQDiHey.jpg   1929-04-20                Alibi   
4    /x1XSQeLUdoi72P5UOXqAyL46B9a.jpg   1928-12-25       In Old Arizona   
..                                ...          ...                  ...   
586  /h7llKkqkkJtJrTOaDLuVeUYDQ7I.jpg   2022-11-11        The Fabelmans   
587  /dRVAlaU0vbG6hMf2K45NSiIyoUe.jpg   2022-09-23                  TÁR   
588  /62HCnUTziyWcpDaBO2i1DX17ljH.jpg   2022-05-24    Top Gun: Maverick   
589  /k9eLozCgCed5FGTSdHu0bBElAV8.jpg   2022-09-18  Triangle of Sadness   
590  /twUbb6Irktv0VEsJXQWJ3VKyxFX.jpg   2022-12-23        Women Talking   

     video  vote_average  vote_count  
0    False         6.400          29  
1    False         7.317          90  
2    False         7.194         247  
3    False         5.656          16  
4    False         5.208          24  
..     ...           ...         ...  
586  False         7.695        1944  
587  False         7.141        1007  
588  False         8.238        8033  
589  False         7.115        1782  
590  False         6.908         465  

[591 rows x 14 columns]

In [32]:
df_movie_data[df_movie_data.isnull().any(axis=1)]

Empty DataFrame
Columns: [adult, backdrop_path, genre_ids, id, original_language, original_title, overview, popularity, poster_path, release_date, title, video, vote_average, vote_count]
Index: []

In [33]:
df_movie_data.drop(columns=["adult", "backdrop_path", "id", "poster_path", "video", "vote_count"], inplace=True)

In [36]:
df_movie_data.drop(columns="popularity", inplace=True)

In [37]:
df_movie_data

genre_ids original_language       original_title  \
0                  [80, 18]                en           The Racket   
1               [18, 10749]                en           7th Heaven   
2    [18, 28, 10752, 10749]                en                Wings   
3              [28, 80, 53]                en                Alibi   
4                  [28, 37]                en       In Old Arizona   
..                      ...               ...                  ...   
586                [18, 35]                en        The Fabelmans   
587             [18, 10402]                en                  TÁR   
588                [28, 18]                en    Top Gun: Maverick   
589                [35, 18]                en  Triangle of Sadness   
590                [18, 80]                en        Women Talking   

                                              overview release_date  \
0    A renegade police captain sets out to catch a ...   1928-11-01   
1    A dejected Parisian sewer worker feels his pra...   1927-09-10   
2    Two young men, one rich, one middle class, bot...   1927-08-12   
3    Chick Williams, a prohibition gangster, rejoin...   1929-04-20   
4    Army Sergeant Mickey Dunn sets out in pursuit ...   1928-12-25   
..                                                 ...          ...   
586  Growing up in post-World War II era Arizona, y...   2022-11-11   
587  Renowned musician Lydia Tár is days away from ...   2022-09-23   
588  After more than thirty years of service as one...   2022-05-24   
589  A celebrity model couple are invited on a luxu...   2022-09-18   
590  A group of women in an isolated religious colo...   2022-12-23   

                   title  vote_average  
0             The Racket         6.400  
1             7th Heaven         7.317  
2                  Wings         7.194  
3                  Alibi         5.656  
4         In Old Arizona         5.208  
..                   ...           ...  
586        The Fabelmans         7.695  
587                  TÁR         7.141  
588    Top Gun: Maverick         8.238  
589  Triangle of Sadness         7.115  
590        Women Talking         6.908  

[591 rows x 7 columns]

In [39]:
df_movie_data['release_date'] = pd.to_datetime(df_movie_data['release_date'])

In [41]:
df_movie_data

genre_ids original_language       original_title  \
0                  [80, 18]                en           The Racket   
1               [18, 10749]                en           7th Heaven   
2    [18, 28, 10752, 10749]                en                Wings   
3              [28, 80, 53]                en                Alibi   
4                  [28, 37]                en       In Old Arizona   
..                      ...               ...                  ...   
586                [18, 35]                en        The Fabelmans   
587             [18, 10402]                en                  TÁR   
588                [28, 18]                en    Top Gun: Maverick   
589                [35, 18]                en  Triangle of Sadness   
590                [18, 80]                en        Women Talking   

                                              overview release_date  \
0    A renegade police captain sets out to catch a ...   1928-11-01   
1    A dejected Parisian sewer worker feels his pra...   1927-09-10   
2    Two young men, one rich, one middle class, bot...   1927-08-12   
3    Chick Williams, a prohibition gangster, rejoin...   1929-04-20   
4    Army Sergeant Mickey Dunn sets out in pursuit ...   1928-12-25   
..                                                 ...          ...   
586  Growing up in post-World War II era Arizona, y...   2022-11-11   
587  Renowned musician Lydia Tár is days away from ...   2022-09-23   
588  After more than thirty years of service as one...   2022-05-24   
589  A celebrity model couple are invited on a luxu...   2022-09-18   
590  A group of women in an isolated religious colo...   2022-12-23   

                   title  vote_average  
0             The Racket         6.400  
1             7th Heaven         7.317  
2                  Wings         7.194  
3                  Alibi         5.656  
4         In Old Arizona         5.208  
..                   ...           ...  
586        The Fabelmans         7.695  
587                  TÁR         7.141  
588    Top Gun: Maverick         8.238  
589  Triangle of Sadness         7.115  
590        Women Talking         6.908  

[591 rows x 7 columns]

In [40]:
df_movie_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 591 entries, 0 to 590
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   genre_ids          591 non-null    object        
 1   original_language  591 non-null    object        
 2   original_title     591 non-null    object        
 3   overview           591 non-null    object        
 4   release_date       591 non-null    datetime64[ns]
 5   title              591 non-null    object        
 6   vote_average       591 non-null    float64       
dtypes: datetime64[ns](1), float64(1), object(5)
memory usage: 32.4+ KB


In [51]:
df_movie_data[df_movie_data.duplicated(subset='title', keep=False)]

genre_ids original_language                  original_title  \
8            [18, 10752]                en  All Quiet on the Western Front   
31           [18, 10751]                en                    Little Women   
37       [36, 18, 10749]                en                       Cleopatra   
53              [18, 36]                fr                  Les Misérables   
56          [12, 18, 36]                en            Mutiny on the Bounty   
65           [18, 10749]                en                Romeo and Juliet   
79           [18, 10749]                en                  A Star Is Born   
132  [14, 18, 35, 10749]                en                 Heaven Can Wait   
229      [80, 18, 10749]                en                 West Side Story   
233         [12, 18, 36]                en            Mutiny on the Bounty   
236      [18, 36, 10749]                en                       Cleopatra   
264          [18, 10749]                en                Romeo and Juliet   
312      [14, 35, 10749]                en                 Heaven Can Wait   
498             [36, 18]                en                  Les Misérables   
552   [10402, 18, 10749]                en                  A Star Is Born   
558          [18, 10749]                en                    Little Women   
580      [18, 10749, 80]                en                 West Side Story   
581          [18, 10752]                de          Im Westen nichts Neues   

                                              overview release_date  \
8    A young soldier faces profound disillusionment...   1930-04-29   
31   Little Women is a coming-of-age drama tracing ...   1933-11-24   
37   The queen of Egypt barges the Nile and flirts ...   1934-10-05   
53   The lives of numerous people over the course o...   1934-02-09   
56   Fletcher Christian successfully leads a revolt...   1935-11-22   
65   Young love is poisoned by a generations long f...   1936-09-03   
79   Esther Blodgett is just another starry-eyed fa...   1937-04-27   
132  Spoiled playboy Henry van Cleve dies and arriv...   1943-08-05   
229  In the slums of the upper West Side of Manhatt...   1961-12-13   
233  The Bounty leaves Portsmouth in 1787. Its dest...   1962-11-08   
236  Determined to hold on to the throne, Cleopatra...   1963-06-12   
264  Romeo Montague and Juliet Capulet fall in love...   1968-04-02   
312  Joe Pendleton is a quarterback preparing to le...   1978-06-28   
498  An adaptation of the successful stage musical ...   2012-12-18   
552  Seasoned musician Jackson Maine discovers — an...   2018-10-03   
558  Four sisters come of age in America in the aft...   2019-12-25   
580  Two youngsters from rival New York City gangs ...   2021-12-08   
581  Paul Baumer and his friends Albert and Muller,...   2022-10-07   

                              title  vote_average  
8    All Quiet on the Western Front         7.734  
31                     Little Women         6.700  
37                        Cleopatra         6.000  
53                   Les Misérables         7.798  
56             Mutiny on the Bounty         7.346  
65                 Romeo and Juliet         5.979  
79                   A Star Is Born         7.157  
132                 Heaven Can Wait         6.872  
229                 West Side Story         7.257  
233            Mutiny on the Bounty         7.013  
236                       Cleopatra         7.055  
264                Romeo and Juliet         7.351  
312                 Heaven Can Wait         6.561  
498                  Les Misérables         7.339  
552                  A Star Is Born         7.500  
558                    Little Women         7.895  
580                 West Side Story         7.024  
581  All Quiet on the Western Front         7.735